# Lagrangian particle tracking

Materials and particles that tend to follow ocean currents can be analyzed in terms of trajectories. These are simply computed by integrating velocities over time within a [Lagrangian framework](https://en.wikipedia.org/wiki/Lagrangian_and_Eulerian_specification_of_the_flow_field). 

In `Julia` this is easily done e.g. using the [IndividualDisplacements.jl](https://JuliaClimate.github.io/IndividualDisplacements.jl/dev/) package.

## 1. Software, Grid, And Background Map

In [1]:
using IndividualDisplacements, DataFrames, Statistics, CSV, Plots
pth=dirname(pathof(IndividualDisplacements))
include(joinpath(pth,"../examples/helper_functions.jl"));

In [2]:
IndividualDisplacements.get_ecco_velocity_if_needed() #download data if needed

ùëÉ,ùê∑=global_ocean_circulation(k=20,ny=2); #grid etc

In [3]:
lon=[i for i=-179.:2.0:179., j=-89.:2.0:89.]
lat=[j for i=-179.:2.0:179., j=-89.:2.0:89.]

(f,i,j,w)=InterpolationFactors(ùê∑.Œì,vec(lon),vec(lat))
Œª=(lon=lon,lat=lat,f=f,i=i,j=j,w=w);

OceanDepth=Interpolate(ùê∑.Œì["Depth"],Œª.f,Œª.i,Œª.j,Œª.w)
OceanDepth=reshape(OceanDepth,size(lon));

## 2. Initialize Individuals

In [4]:
ùê∑.üîÑ(ùëÉ,ùê∑,0.0) #update velocity fields (here, to Dec and Jan bracketing t=0.0)

np=100
xy = init_global_randn(np,ùê∑)
df=DataFrame(x=xy[1,:],y=xy[2,:],f=xy[3,:]) #initial positions

ùêº=Individuals(ùëÉ,df.x[1:np],df.y[1:np],df.f[1:np]) #Individuals data structure

fieldnames(typeof(ùêº))

(:üìå, :üî¥, :üÜî, :üöÑ, :‚à´, :üîß, :ùëÉ, :ùê∑, :ùëÄ)

## 3. Compute Trajectories

In [5]:
ùëá=(0.0,ùêº.ùëÉ.ùëá[2]) #first half of January
‚à´!(ùêº,ùëá) #mid-Dec to mid-Jan

for m=1:12
    ùê∑.üîÑ(ùëÉ,ùê∑,0.0) #update velocity fields
    ‚à´!(ùêº) #integrate forward by one more month
end

## 4. Post-Processing

In [6]:
add_lonlat!(ùêº.üî¥,ùê∑.XC,ùê∑.YC)
ùêº.üî¥[end-3:end,:]

4√ó7 DataFrame
 Row ‚îÇ ID     x         y         fid    t        lon        lat      
     ‚îÇ Int64  Float64   Float64   Int64  Float64  Float64    Float64  
‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îº‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ‚îÄ
   1 ‚îÇ    97   85.0896  200.97        2  1.314e6   137.09     29.6546
   2 ‚îÇ    98   34.0998   50.9719      4  1.314e6  -167.028    54.7692
   3 ‚îÇ    99  122.976    15.2586      4  1.314e6   157.259   -12.8848
   4 ‚îÇ   100   36.5861   98.4732      2  1.314e6    88.5861  -52.2346

## 5. Plot Trajectories

In this example we simply map out individual positions (red to blue) over a map of ocean depth (log10).

In [7]:
plt=contourf(vec(lon[:,1]),vec(lat[1,:]),permutedims(OceanDepth),clims=(-2500.,5000.),c = :grays, colorbar=false)
xli=extrema(lon)

üî¥_by_t = groupby(ùêº.üî¥, :t)
lo=deepcopy(üî¥_by_t[1].lon); lo[findall(lo.<xli[1])]=lo[findall(lo.<xli[1])].+360
scatter!(lo,üî¥_by_t[1].lat,markersize=1.5,c=:red,leg=:none,marker = (:circle, stroke(0)))
lo=deepcopy(üî¥_by_t[end].lon); lo[findall(lo.<xli[1])]=lo[findall(lo.<xli[1])].+360
scatter!(lo,üî¥_by_t[end].lat,markersize=1.5,c=:blue,leg=:none,marker = (:dot, stroke(0)))